In [39]:
import pandas as pd
y = [10,17,16,15,22,31,8,45]
x = [2,3,4,5,6,7,8,9]
#y = [12,14.4,14.6,16,11.3,10,16.2,10.4,13.1,11.3]
#x = [12,21,18,22,13,10,23,11,16,14]
df = pd.DataFrame({"Y":y,"X":x})
df.index+=1
df.T

,1,2,3,4,5,6,7,8
Y,10,17,16,15,22,31,8,45
X,2,3,4,5,6,7,8,9


### Least Squares Method

In [40]:
import matplotlib.pyplot as plt
import numpy as np
y = np.matrix(y).T #dependent variable 
x1 = x #independent variable

x = np.matrix([np.ones(len(y)),x1]).T
xx = np.dot(x.T,x)
xinv = np.linalg.inv(xx) #(x'x)-1
xy = np.dot(x.T,y)  #x'y
beta = xinv*xy

b0 = np.round(beta[0].item(0),4)
b1 = np.round(beta[1].item(0),4)

print(f"x\n{x}\n\nx'\n{x.T}\n\nx'x\n{xx}\n\ny\n{y}\n\nx'y\n{xy}\n\n(x'x)^-1\n{xinv}\n\n((x'x)^-1)*(x'y)\n{beta}",
f"\n\nY predicted model = {b0} + {b1} * x1 ")

predicts = list(map(lambda x:b0+b1*x,x1))
predicts = np.array(np.round(predicts,4))
print("\nPredicted values = ",predicts)

err = np.zeros(len(y))
n = len(y)

for i in range(len(y)):
    
    err[i]=y[i]-predicts[i]

errs = list(map(lambda i :i**2,err))
print(f"\nErrors = {err}\n\nError's Squares = {errs} ")

x
[[1. 2.]
 [1. 3.]
 [1. 4.]
 [1. 5.]
 [1. 6.]
 [1. 7.]
 [1. 8.]
 [1. 9.]]

x'
[[1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 3. 4. 5. 6. 7. 8. 9.]]

x'x
[[  8.  44.]
 [ 44. 284.]]

y
[[10]
 [17]
 [16]
 [15]
 [22]
 [31]
 [ 8]
 [45]]

x'y
[[ 164.]
 [1028.]]

(x'x)^-1
[[ 0.8452381  -0.13095238]
 [-0.13095238  0.02380952]]

((x'x)^-1)*(x'y)
[[4.]
 [3.]] 

Y predicted model = 4.0 + 3.0 * x1 

Predicted values =  [10. 13. 16. 19. 22. 25. 28. 31.]

Errors = [  0.   4.   0.  -4.   0.   6. -20.  14.]

Error's Squares = [0.0, 16.0, 0.0, 16.0, 0.0, 36.0, 400.0, 196.0] 


### Standardization Residuals Analysis

In [41]:
sse =np.sum(list(map(lambda x : x**2,err)))

mse = sse/(len(y)-1)
sigma = np.sqrt(mse)
d = []
for i in err:
    d.append(i/sigma)
df2 = pd.DataFrame(d)
df2.index+=1
print(df2.T)
print(f"\nMean Squared Error = {mse}")
for i in d:
    if i>=2 or i<=-2:
        print(f"\n{np.round(i,4)} is should examine")
        

     1       2    3       4    5        6         7         8
0  0.0  0.4107  0.0 -0.4107  0.0  0.61605 -2.053501  1.437451

Mean Squared Error = 94.85714285714286

-2.0535 is should examine


### Heteroskedasticity Problem

In [42]:
df["rank(x)"] = df["X"].rank()
df["predicts"] = predicts
df["errors"] = np.abs(err)
df["rank(e)"] = df["errors"].rank()
df["di"] = df["rank(e)"]-df["rank(x)"]
df["di^2"] = df["di"]**2
df.T

,1,2,3,4,5,6,7,8
Y,10.0,17.00,16.0,15.00,22.0,31.0,8.0,45.0
X,2.0,3.00,4.0,5.00,6.0,7.0,8.0,9.0
rank(x),1.0,2.00,3.0,4.00,5.0,6.0,7.0,8.0
predicts,10.0,13.00,16.0,19.00,22.0,25.0,28.0,31.0
errors,0.0,4.00,0.0,4.00,0.0,6.0,20.0,14.0
rank(e),2.0,4.50,2.0,4.50,2.0,6.0,8.0,7.0
di,1.0,2.50,-1.0,0.50,-3.0,0.0,1.0,-1.0
di^2,1.0,6.25,1.0,0.25,9.0,0.0,1.0,1.0


In [43]:
rs = 1 - ((6*df["di^2"].sum())/(n*(n**2)-n))
rs

0.7678571428571428

$$H_0 : \rho = 0$$<br>$$H_s : \rho \neq 0$$

In [44]:
t = np.round(((rs)*(np.sqrt(n-2)))/(np.sqrt(1-rs**2)),3)
t

2.936

In [45]:
import scipy.stats as ss
th = np.round(ss.t.ppf(q=0.05,df=n-2),3)#tablo
if th>t:
    print(f"th({th}) > t({t}). You fail to reject the null hypothesis. There is not heteroskedasticity.")
else:
    print(f"th({th}) < t({t}). You can reject the null hypothesis. There is heteroskedasticity. ")


th(-1.943) < t(2.936). You can reject the null hypothesis. There is heteroskedasticity. 


### Autocorrelation

In [34]:
et = []
for i in range(n):
    et.append(err[i]*err[i-1])
df["etet_1"] = et
df

,Y,X,rank(x),predicts,errors,rank(e),di,di^2,etet_1
1,10,2,1.0,10.0,0.0,4.0,3.0,9.0,0.0
2,17,3,2.0,13.0,4.0,6.0,4.0,16.0,0.0
3,16,4,3.0,16.0,0.0,4.0,1.0,1.0,0.0
4,15,5,4.0,19.0,-4.0,2.0,-2.0,4.0,-0.0
5,22,6,5.0,22.0,0.0,4.0,-1.0,1.0,-0.0
6,31,7,6.0,25.0,6.0,7.0,1.0,1.0,0.0
7,8,8,7.0,28.0,-20.0,1.0,-6.0,36.0,-120.0
8,45,9,8.0,31.0,14.0,8.0,0.0,0.0,-280.0


In [46]:
sse

664.0

In [36]:
rho = df["etet_1"].sum()/sse
rho

-0.6024096385542169

There is negative autocorrelation